In [ ]:
class CharLevel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, Tx, m, w, N):
        super(CharLevel, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embedding_dim)
        #what's w?
        #what's N? Where do we define all the scalar values needed for this?
        #emb_dim is called d_c in the paper
        
        
        """I think I'm most confused about the convolution part. The paper doesn't use the same
        vocabulary as the documentaton for torch's built-in convolution layers, and I'm not sure sure
        if the built-in convolution options are enough to do it the same way as the paper"""
        #this seems helpful:
        #https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
        
        #do we pad here or in the convolution layer? Both have a padding option.
        
        # I think filtering isn't a separate layer because we can filter using the parameters.
        # I think the kernel is the "filter" and "stride" controls how the filter convolves around the input volume.
        # Not sure about "groups" which is the "Number of blocked connections from input channels to output channels."
        
        #are there multiple conv layers for different ngram lengths so we can perform this on all ngrams 
        #up to 8 characters long or are we feeding into the model every length of ngram at once?
        #so the preprocessing steps would need to preprocess the data with ngrams from 1-8?
        
        #not sure if I have Tx, N, w all in the right place for the parameter values
        
        self.convolve = nn.Conv1d(Tx, N, w, padding=w-1, bias=False)
        
        self.relu = nn.ReLU()
        
        #I think both the kernel and the stride are size 5 so that it's non-overlapping
        self.maxpool = nn.MaxPool1d(kernel = 5, stride=5)
        
        #this is all part of the "highway"
        #got the inspiration for this from
        #https://github.com/c0nn3r/pytorch_highway_networks/blob/master/layers/highway.py
        self.activation_function = nn.functional.relu
        self.gate_activation = nn.functional.softmax #I think the paper doesn't say
        #whether this is softmax or something else - could be sigmoid?

        self.normal_layer = nn.Linear(input_size, input_size)

        self.gate_layer = nn.Linear(input_size, input_size)
        self.gate_layer.bias.data.fill_(gate_bias)
        
        #done with the highway
        
        self.GRU = nn.GRU(bidirectional=True)
        self.init_weights()
        
                
    def forward(self, x, hidden, c):
        """What goes in the "forward" and what goes in the "step"?
        "Step" has to be called in the "forward" so I think it doesn't matter, it's just to keep things organized"""
        #Embedding
        x_emb = self.encoder(x)
        x_conv = self.convolve(x_emb)
        x_relu = self.relu(x_conv)
        x_MaxPool = self.maxpool(x_relu)
        ###this is all for the "highway"
        normal_layer_result = self.activation_function(self.normal_layer(x_MaxPool))
        gate_layer_result = self.gate_activation(self.gate_layer(x_MaxPool))

        multiplyed_gate_and_normal = torch.mul(normal_layer_result, gate_layer_result)
        multiplyed_gate_and_input = torch.mul((1 - gate_layer_result), x_MaxPool)

        x_highway = torch.add(multiplyed_gate_and_normal,
                         multiplyed_gate_and_input)
    
        return self.GRU(x_highway)
        
    def init_weights(self):
        """Blabla not sure what goes here"""
                
    def init_hidden(self, bsz):
        """Blabla I don't think we need this"""